In [6]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import sys
import os
#sys.path.append('/Users/ff/dev/MSc/CX_Path_Integration')
#os.chdir('/Users/ff/dev/MSc/CX_Path_Integration/notebooks')

In [183]:
from brian2 import *
from brian2tools import *

import matplotlib.pyplot as plt

import cx_rate
import trials
import plotter

import cx_spiking.plotting 
import cx_spiking.inputs
import cx_spiking.optimisation.metric as metric

In [8]:
# Constants
N_TL2 = 16
N_CL1 = 16
N_TB1 = 8
N_TN2 = 2

In [9]:
route_file = 'data/route.npz'
T_outbound = 1500

h, v, = cx_spiking.inputs.generate_route(T_outbound=T_outbound, vary_speed=True, route_file=route_file, load_route=True)

cx_spiking.inputs.save_route(route_file, h, v, save_route=True)

# Convert headings
headings, digitized = cx_spiking.inputs.compute_headings(h, N=N_TL2//2, vmin=5, vmax=100)
headings = np.tile(headings, 2)

# Convert velocity into optical flow
flow = cx_spiking.inputs.compute_flow(h, v, baseline=50, vmin=0, vmax=50)

Load route from data/route.npz
data/route.npz exists - not overwriting it


In [10]:
noise = 0.1
cx = cx_rate.CXRatePontin(noise=noise)

h, v, log, cpu4_snapshot = trials.run_trial(logging=True,
                                            T_outbound=T_outbound,
                                            T_inbound=0,
                                            noise=noise,
                                            cx=cx,
                                            route=(h[:T_outbound], v[:T_outbound]))

In [13]:
start_scope()
#defaultclock.dt = 0.05*ms

time_step = 20 # ms

recorded_spike_rates = 90 # Hz
ratio = recorded_spike_rates / (time_step * 1./1000)
print(f'ratio {ratio} - dt {defaultclock.dt}')

ratio 4500.0 - dt 0.0001


In [14]:
# transpose since log is neuron_index*time_step but we want the opposite
TL2_stimulus = TimedArray(100*log.tl2.T*Hz, dt=1.*time_step*ms)
P_TL2 = PoissonGroup(N_TL2, rates='TL2_stimulus(t,i)')
SPM_TL2_IDEAL = SpikeMonitor(P_TL2)

P_TL2_2 = PoissonGroup(N_TL2, rates='TL2_stimulus(t,i)')
SPM_TL2_2_IDEAL = SpikeMonitor(P_TL2_2)

In [15]:
%%time
run(T_outbound*time_step*ms)

CPU times: user 20 s, sys: 542 ms, total: 20.5 s
Wall time: 30.6 s


In [201]:
gf = metric.compute_gamma_factor(SPM_TL2_2_IDEAL, SPM_TL2_2_IDEAL, T_outbound*time_step*ms, 
                                 dt_=defaultclock.dt, delta=1*ms, rate_correction=True)
print(f'mean gamma factor {gf}')

mean gamma factor 2.7755575615628914e-17
